In [2]:
!pip install -Uqq fastai # upgrade fastai

In [3]:
from fastai.tabular.all import *

path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/home/weidong/.fastai/data/adult_sample/export.pkl'),Path('/home/weidong/.fastai/data/adult_sample/models'),Path('/home/weidong/.fastai/data/adult_sample/adult.csv')]

In [4]:
df = pd.read_csv(path / 'adult.csv')

In [5]:
df.head(20)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k
5,20,Private,63210,HS-grad,9.0,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,15,United-States,<50k
6,49,Private,44434,Some-college,10.0,Divorced,NaN,Other-relative,White,Male,0,0,35,United-States,<50k
7,37,Private,138940,11th,7.0,Married-civ-spouse,NaN,Husband,White,Male,0,0,40,United-States,<50k
8,46,Private,328216,HS-grad,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>=50k
9,36,Self-emp-inc,216711,HS-grad,NaN,Married-civ-spouse,NaN,Husband,White,Male,99999,0,50,?,>=50k


In [7]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                  cat_names=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                                  cont_names=['age', 'fnlwgt', 'education-num'],
                                  procs=[Categorify, FillMissing, Normalize])

/home/weidong/develop/python-playground/venv/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Some-college,Married-civ-spouse,Adm-clerical,Husband,Black,False,29.000000,231553.999505,10.0,<50k
1,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,False,40.000000,192380.999908,10.0,>=50k
2,Private,HS-grad,Widowed,#na#,Unmarried,White,True,68.000001,223486.000454,10.0,<50k
3,Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,33.000000,222162.000909,9.0,<50k
4,Private,Assoc-acdm,Never-married,Other-service,Own-child,White,False,21.000000,177264.999949,12.0,<50k
5,Private,HS-grad,Never-married,Other-service,Other-relative,Amer-Indian-Eskimo,False,33.000000,110998.000458,9.0,<50k
6,Local-gov,Some-college,Married-spouse-absent,Craft-repair,Other-relative,White,False,47.000000,162595.001506,10.0,<50k
7,Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,White,False,61.000000,130683.998231,10.0,<50k
8,Private,9th,Divorced,Adm-clerical,Not-in-family,White,False,31.000000,118550.999211,5.0,<50k
9,Private,10th,Never-married,Craft-repair,Not-in-family,White,False,22.000000,186451.999946,6.0,<50k


In [21]:
print(f"Total training samples: {len(dls.train_ds)}")
print(f"Total validation samples: {len(dls.valid_ds)}")

x_cat, x_cont, y = dls.one_batch()
print(f"Categorical Input Shape: {x_cat.shape}")  # (batch_size, num_cat_features)
print(f"Continuous Input Shape: {x_cont.shape}")  # (batch_size, num_cont_features)
print(f"Target Shape: {y.shape}")  # (batch_size,)

print("Categorical features (batch):", x_cat[:5])  # First 5 samples
print("Continuous features (batch):", x_cont[:5])
print("Target values (batch):", y[:5])

Total training samples: 26049
Total validation samples: 6512
Categorical Input Shape: torch.Size([64, 7])
Continuous Input Shape: torch.Size([64, 3])
Target Shape: torch.Size([64, 1])
Categorical features (batch): tensor([[ 5,  2,  5,  9,  4,  5,  1],
        [ 5,  1,  5,  9,  4,  5,  1],
        [ 8, 16,  3,  8,  1,  5,  1],
        [ 2, 13,  3,  5,  1,  5,  1],
        [ 5, 12,  3,  8,  1,  5,  1]])
Continuous features (batch): tensor([[-1.5837, -0.2069, -1.2002],
        [-1.5837,  0.7035, -1.5918],
        [ 0.7611, -0.6409, -0.0253],
        [ 0.1749,  0.2390,  1.5411],
        [ 0.1016, -1.0748, -0.4169]])
Target values (batch): tensor([[0],
        [0],
        [1],
        [1],
        [0]], dtype=torch.int8)


In [10]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Self-emp-not-inc,Masters,Married-civ-spouse,Exec-managerial,Husband,White,False,40.0,204235.000616,14.0,>=50k
1,Private,Prof-school,Married-civ-spouse,Prof-specialty,Husband,White,False,40.0,77357.001028,15.0,>=50k
2,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,False,32.0,110330.998184,9.0,<50k
3,Private,HS-grad,Divorced,Other-service,Not-in-family,White,False,53.0,139126.999722,9.0,<50k
4,Private,Some-college,Married-civ-spouse,Other-service,Husband,White,False,27.0,197904.999682,10.0,<50k
5,Private,HS-grad,Divorced,Prof-specialty,Not-in-family,White,False,31.0,127595.000714,9.0,<50k
6,State-gov,HS-grad,Married-civ-spouse,Protective-serv,Husband,White,False,32.0,175931.000109,9.0,<50k
7,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,False,43.0,75742.002778,9.0,<50k
8,Private,Some-college,Married-civ-spouse,Other-service,Husband,White,False,27.0,197904.999682,10.0,<50k
9,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,False,54.0,36479.999495,9.0,>=50k


In [15]:
learn = tabular_learner(dls, metrics=accuracy)

In [16]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.375434,0.361829,0.837070,00:02


In [17]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,6.0,10.0,3.0,5.0,1.0,5.0,1.0,0.174880,0.295848,1.149488,1.0,1.0
1,8.0,10.0,3.0,11.0,1.0,5.0,1.0,-0.777668,-0.380255,1.149488,1.0,1.0
2,3.0,10.0,5.0,11.0,2.0,2.0,1.0,-1.070760,-1.475911,1.149488,0.0,0.0
3,1.0,8.0,5.0,1.0,2.0,5.0,1.0,-1.363851,-0.004404,0.757882,0.0,0.0
4,5.0,16.0,5.0,8.0,2.0,3.0,1.0,-0.264757,-0.222513,-0.025331,0.0,0.0
5,5.0,1.0,5.0,9.0,4.0,5.0,1.0,-1.583670,0.021303,-1.591758,0.0,0.0
6,5.0,16.0,3.0,5.0,1.0,5.0,1.0,0.467972,2.466050,-0.025331,0.0,1.0
7,5.0,16.0,5.0,2.0,5.0,3.0,1.0,-0.777668,-0.032551,-0.025331,0.0,0.0
8,5.0,10.0,5.0,13.0,2.0,5.0,1.0,0.321426,-0.700202,1.149488,1.0,0.0


In [18]:
print(f"Categorical Features: {len(dls.cat_names)} -> {dls.cat_names}")
print(f"Continuous Features: {len(dls.cont_names)} -> {dls.cont_names}")
print(f"Target Column: {dls.y_names}")

Categorical Features: 7 -> ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'education-num_na']
Continuous Features: 3 -> ['age', 'fnlwgt', 'education-num']
Target Column: ['salary']


In [22]:
row, clas, probs = learn.predict(df.iloc[0])

/home/weidong/develop/python-playground/venv/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [33]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

/home/weidong/develop/python-playground/venv/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [34]:
learn.get_preds(dl=dl)

(tensor([[0.8034, 0.1966],
         [0.5040, 0.4960],
         [0.8342, 0.1658],
         ...,
         [0.7885, 0.2115],
         [0.8878, 0.1122],
         [0.9203, 0.0797]]),
 None)